In [1]:
import json
from sentence_transformers import SentenceTransformer, util
import sys
import pandas as pd
sys.path.append('../')
from pandarec.panda_rec_widget import PandaRecWidget
from pandarec.recipe import Recipe, RecipeResult
from pandarec.recommender import Recommender
from pandarec.strategies import RankingStrategy
from pandarec.context import Context
import torch

In [3]:
with open('../recipes/from_docstrings/recipes.json') as json_file:
    data = json.load(json_file)

In [4]:
recipes = [Recipe.from_dict(recipe) for recipe in data]

In [5]:
with open("./cars.json") as file:
    data = json.load(file)

df = pd.DataFrame(data["data"]).drop("index", axis=1)

In [6]:
class SemanticSearch(RankingStrategy):
    def __init__(self, recipes: list[Recipe], embeddings_path: str = None):
        super().__init__()
        self.model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
        descriptions = [recipe.description for recipe in recipes]
        if not embeddings_path:
            self.embeddings = self.model.encode(descriptions, convert_to_tensor=True)
        else:
            self.load_embeddings(embeddings_path)

    def search(self, context: Context, recipes: list[Recipe]) -> list[RecipeResult]:
        query_embedding = self.model.encode(context.query, convert_to_tensor=True)
        cos_scores = [utils.cos_sim(query_embedding, embedding) for embedding in self.embeddings]
        results = [RecipeResult(score, recipe) for recipe, score in zip(recipes, cos_scores)]
        return results

    def save_embeddings(self, path):
        torch.save(self.embeddings, path)

    def load_embeddings(self, path):
        self.embeddings = torch.load(path)


In [8]:
SemanticSearch(recipes).search(Context([], None, "chicken"), recipes)

KeyboardInterrupt: 